In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 

plt.style.use('fivethirtyeight')

# 노트북 안에 그래프 그리기
%matplotlib inline
# 그래프 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일 사용
plt.style.use('ggplot')
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus']=False

# 컬럼을 다 보여주는 함수
from IPython.display import display
pd.options.display.max_columns = None

# 1. 데이터 불러오기

In [21]:
com_live_pop = pd.read_csv('상권_상주인구.csv',engine='python',encoding='utf-8',parse_dates=['기준_년월_코드'])
print(com_live_pop.shape)
print("-----------------------------")
com_income = pd.read_csv('상권_소득소비.csv',engine='python',encoding='utf-8',parse_dates=['기준_년월_코드'])
print(com_income.shape)
print("-----------------------------")
com_work_pop = pd.read_csv('상권_직장인구.csv',engine='python',encoding='utf-8',parse_dates=['기준_년월_코드'])
print(com_work_pop.shape)
print("-----------------------------")
com_fac = pd.read_csv('상권_집객시설.csv',engine='python',encoding='utf-8',parse_dates=['기준_년월_코드'])
print(com_fac.shape)
print("-----------------------------")
com_sales = pd.read_csv('상권_추정매출.csv',engine='python',encoding='utf-8',parse_dates=['기준_년월_코드'])
print(com_sales.shape)
print("-----------------------------")
com_fpop = pd.read_csv('상권_추정유동인구.csv',engine='python',encoding='utf-8',parse_dates=['기준_년월_코드'])
print(com_fpop.shape)

(36128, 27)
-----------------------------
(35740, 15)
-----------------------------
(36288, 24)
-----------------------------
(23088, 23)
-----------------------------
(1120485, 83)
-----------------------------
(36228, 25)


# 2. 데이터 정제

- 상권추정매출

In [22]:
com_sales.isnull().sum()

기준_년월_코드                0
상권_코드                   0
상권_코드_명              3818
서비스_업종_코드               0
서비스_업종_코드_명             0
평균_영업_개월_수         505483
1년이하_생존_율          793405
1~2년_생존_율          793405
2~3년_생존_율          793405
3~5년_생존_율          793405
5년이상_생존_율          793405
당월_매출_금액                0
당월_매출_건수                0
주중_매출_비율                0
주말_매출_비율                0
월요일_매출_비율               0
화요일_매출_비율               0
수요일_매출_비율               0
목요일_매출_비율               0
금요일_매출_비율               0
토요일_매출_비율               0
일요일_매출_비율               0
시간대_00~06_매출_비율         0
시간대_06~11_매출_비율         0
시간대_11~14_매출_비율         0
시간대_14~17_매출_비율         0
시간대_17~21_매출_비율         0
시간대_21~24_매출_비율         0
남성_매출_비율                0
여성_매출_비율                0
                    ...  
연령대_10_매출_금액            0
연령대_20_매출_금액            0
연령대_30_매출_금액            0
연령대_40_매출_금액            0
연령대_50_매출_금액            0
연령대_60_이상_매출_금액         0
주중_매출_건수                0
주말_매출_건수    

In [23]:
com_sales[com_sales['상권_코드_명'].isnull()].iloc[:,:10][6:11]

,기준_년월_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,평균_영업_개월_수,1년이하_생존_율,1~2년_생존_율,2~3년_생존_율,3~5년_생존_율
8263,201301,12621,NaN,CS100007,치킨집,NaN,NaN,NaN,NaN,NaN
8264,201301,12621,NaN,CS100010,호프간이주점,NaN,NaN,NaN,NaN,NaN
8265,201301,12621,NaN,CS200003,예체능학원,NaN,NaN,NaN,NaN,NaN
8266,201301,12621,NaN,CS200004,치과의원,NaN,NaN,NaN,NaN,NaN
8267,201301,12621,NaN,CS200006,일반의원,NaN,NaN,NaN,NaN,NaN


In [24]:
com_sales = com_sales[com_sales['기준_년월_코드']>=201501]
com_sales.head()
# 상권_코드_명 의 결측치 제거
com_sales = com_sales[com_sales['상권_코드_명'].notnull()]
com_sales.isnull().sum()

기준_년월_코드                0
상권_코드                   0
상권_코드_명                 0
서비스_업종_코드               0
서비스_업종_코드_명             0
평균_영업_개월_수         115621
1년이하_생존_율          328661
1~2년_생존_율          328661
2~3년_생존_율          328661
3~5년_생존_율          328661
5년이상_생존_율          328661
당월_매출_금액                0
당월_매출_건수                0
주중_매출_비율                0
주말_매출_비율                0
월요일_매출_비율               0
화요일_매출_비율               0
수요일_매출_비율               0
목요일_매출_비율               0
금요일_매출_비율               0
토요일_매출_비율               0
일요일_매출_비율               0
시간대_00~06_매출_비율         0
시간대_06~11_매출_비율         0
시간대_11~14_매출_비율         0
시간대_14~17_매출_비율         0
시간대_17~21_매출_비율         0
시간대_21~24_매출_비율         0
남성_매출_비율                0
여성_매출_비율                0
                    ...  
연령대_10_매출_금액            0
연령대_20_매출_금액            0
연령대_30_매출_금액            0
연령대_40_매출_금액            0
연령대_50_매출_금액            0
연령대_60_이상_매출_금액         0
주중_매출_건수                0
주말_매출_건수    

In [25]:
# 평균 업업 개월 수 ,생존율 drop
com_sales.drop(['평균_영업_개월_수', '1년이하_생존_율', '1~2년_생존_율', '2~3년_생존_율', '3~5년_생존_율','5년이상_생존_율'],axis=1,inplace=True)

In [26]:
# 상권상주인구
com_live_pop = com_live_pop[['기준_년월_코드','상권_코드','상권_코드_명','총_상주인구_수','총_가구_수','아파트_가구_수','비_아파트_가구_수']]
com_live_pop = com_live_pop[com_live_pop['기준_년월_코드']>=201501].sort_values(by=['기준_년월_코드','상권_코드'])
com_live_pop = com_live_pop[com_live_pop['상권_코드_명'].notnull()]
com_live_pop.isnull().sum()
com_live_pop.head()

,기준_년월_코드,상권_코드,상권_코드_명,총_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수
13044,201501,11947,당산로44길,194,84.0,84,0.0
13045,201501,11948,동일로242길,1789,638.0,124,514.0
13046,201501,11949,동일로_A,748,288.0,173,115.0
13047,201501,11950,동일로_B,190,70.0,0,70.0
13048,201501,11953,도봉로_A,2078,890.0,0,890.0


- 상권소득소비

In [27]:
com_income = com_income[['기준_년월_코드','상권_코드','상권_코드_명','월_평균_소득_금액','소득_구간_코드']][com_income['기준_년월_코드']>=201501]
com_income = com_income[com_income['상권_코드_명'].notnull()]
com_income.isnull().sum()
com_income.head()

,기준_년월_코드,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드
11952,201501,11947,당산로44길,460,6
11953,201501,11948,동일로242길,301,3
11954,201501,11949,동일로_A,346,4
11955,201501,11950,동일로_B,235,2
11956,201501,11953,도봉로_A,422,5


- 상권집객시설

In [28]:
com_fac = com_fac[['기준_년월_코드','상권_코드','상권_코드_명','집객시설_수']][com_fac['기준_년월_코드']>=201501]
com_fac = com_fac[com_fac['상권_코드_명'].notnull()]
com_fac.isnull().sum()
com_fac.head()

,기준_년월_코드,상권_코드,상권_코드_명,집객시설_수
0,201501,11947,당산로44길,10
1,201501,11948,동일로242길,11
2,201501,11949,동일로_A,6
3,201501,11950,동일로_B,22
4,201501,11953,도봉로_A,12


- 상권 유동인구

In [29]:
com_fpop = com_fpop[com_fpop['기준_년월_코드']>=201501]
com_fpop = com_fpop[com_fpop['상권_코드_명'].notnull()]
com_fpop.isnull().sum()
com_fpop.iloc[:,:10].head()

,기준_년월_코드,상권_코드,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수
12,201501,12169,용마산로115길_B,5165,2609,2556,175,572,1044,1247
13,201502,12169,용마산로115길_B,4602,2333,2269,159,480,952,1114
14,201503,12169,용마산로115길_B,5913,2987,2926,208,593,1210,1385
15,201504,12169,용마산로115길_B,5709,2885,2824,197,574,1200,1334
16,201505,12169,용마산로115길_B,5909,2951,2958,239,620,1245,1368


- 상권직장인구

In [30]:
com_work_pop = com_work_pop[['기준_년월_코드','상권_코드','상권_코드_명','총_직장_인구_수','남성_직장_인구_수',
                             '여성_직장_인구_수','연령대_10_직장_인구_수','연령대_20_직장_인구_수','연령대_30_직장_인구_수',
                             '연령대_40_직장_인구_수','연령대_50_직장_인구_수','연령대_60_이상_직장_인구_수']][com_work_pop['기준_년월_코드']>=201501]
com_work_pop = com_work_pop[com_work_pop['상권_코드_명'].notnull()]
com_work_pop.isnull().sum()
com_work_pop.iloc[:,:10].head()

,기준_년월_코드,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수
12096,201501,11947,당산로44길,375,148,227,0,30,121,121
12097,201501,11948,동일로242길,2169,1704,465,1,53,165,314
12098,201501,11949,동일로_A,272,114,158,0,20,62,71
12099,201501,11950,동일로_B,485,204,281,0,53,133,129
12100,201501,11953,도봉로_A,886,462,424,0,31,162,268


# 3. 테이블 병합 및 null값 처리

In [31]:
# com_sales, com_live_pop merge
com_df = pd.merge(com_sales,com_live_pop,on=['기준_년월_코드','상권_코드','상권_코드_명'],how='left')

# '총_상주인구_수','총_가구_수','아파트_가구_수','비_아파트_가구_수' 의 null값제거
com_df[['총_상주인구_수','총_가구_수','아파트_가구_수','비_아파트_가구_수']] = com_df[['총_상주인구_수','총_가구_수',
                                                                                '아파트_가구_수','비_아파트_가구_수']].fillna(0)

# com_df, com_income merge
com_df = pd.merge(com_df,com_income,on=['기준_년월_코드','상권_코드','상권_코드_명'],how='left')

# 월_평균_소득_금액이 null값인것들 제거 ex) 노해로_A같은건 매출테이블에는 있지만 소득테이블에는 없음
com_df = com_df[com_df['월_평균_소득_금액'].notnull()]

# com_df, com_fac merge
com_df = pd.merge(com_df,com_fac,on=['기준_년월_코드','상권_코드','상권_코드_명'],how='left')

# fac의 null값은 19000여개가 존재하지만 매출테이블에는 존재하지 않는 상권이므로 제거
com_df = com_df[com_df['집객시설_수'].notnull()]

# com_df, com_fpop merge
com_df = pd.merge(com_df,com_fpop,on=['기준_년월_코드','상권_코드','상권_코드_명'],how='left')

# com_df, com_work_pop merge
com_df = pd.merge(com_df,com_work_pop,on=['기준_년월_코드','상권_코드','상권_코드_명'],how='left')

com_df = com_df[com_df['남성_유동인구_수'].notnull()]
# 업종이 외식업인 데이터만 사용
com_df = com_df[com_df["서비스_업종_코드"]<="CS100010"]


print("*** com_df shape : ", com_df.shape,"***")
print(" ")
com_df.info()

*** com_df shape :  (171131, 115) ***
 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 171131 entries, 0 to 532994
Columns: 115 entries, 기준_년월_코드 to 연령대_60_이상_직장_인구_수
dtypes: float64(52), int64(60), object(3)
memory usage: 151.5+ MB


In [32]:
# csv파일 저장
com_df.to_csv('market.csv',index=False)